In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w7p2/sample_submission.csv
/kaggle/input/2022-ai-w7p2/x_test.csv
/kaggle/input/2022-ai-w7p2/y_train.csv
/kaggle/input/2022-ai-w7p2/x_train.csv


In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
x_train = pd.read_csv("/kaggle/input/2022-ai-w7p2/x_train.csv", encoding='cp949')
y_train = pd.read_csv("/kaggle/input/2022-ai-w7p2/y_train.csv", encoding='cp949')
x_test = pd.read_csv("/kaggle/input/2022-ai-w7p2/x_test.csv", encoding='cp949')
submit = pd.read_csv("/kaggle/input/2022-ai-w7p2/sample_submission.csv")

In [4]:
x_train.columns

Index(['시도명', '읍면동명', '거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도', '평균 소요 시간',
       '평균 기온', '일강수량', '평균 풍속', '월', '일', '년'],
      dtype='object')

In [5]:
x_train = x_train.drop(['시도명', '읍면동명', '월', '일', '년'],axis=1)
x_test = x_test.drop(['시도명', '읍면동명', '월', '일', '년'],axis=1)

In [6]:
x_test.head()

,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속
0,45737.828,8703.034,91775.025,146215.887,57.286,35.000,6.500,0.0,2.9
1,289284.270,48392.749,179649.272,517326.291,50.500,58.912,18.833,0.0,2.3
2,72684.218,45632.065,120931.098,239247.380,34.727,60.000,13.200,0.0,1.8
3,235752.513,34212.805,186910.679,456875.997,37.167,26.500,11.700,0.0,4.8
4,544250.956,64188.293,205182.677,813621.925,31.462,37.769,22.233,0.0,2.3


In [7]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_train = ss.transform(x_train)
x_test = ss.transform(x_test)

In [8]:
y_train = np.array(y_train)

In [9]:
x_train = torch.FloatTensor(x_train).cuda()
y_train = torch.FloatTensor(y_train).cuda()
x_test = torch.FloatTensor(x_test).cuda()

In [10]:
from torch.utils.data import TensorDataset

train_data = TensorDataset(x_train, y_train)

In [11]:
from torch.utils.data import DataLoader
batch_size = 50
data_loader = DataLoader(dataset=train_data,
                        batch_size=batch_size,
                        shuffle=True,
                        drop_last=True)

In [12]:
x_train.shape

torch.Size([21486, 9])

In [13]:
layer1 = torch.nn.Linear(9, 900, bias = True)
layer2 = torch.nn.Linear(900, 700, bias = True)
layer3 = torch.nn.Linear(700, 500, bias = True)
layer4 = torch.nn.Linear(500, 250, bias = True)
layer5 = torch.nn.Linear(250, 1, bias = True)
relu = torch.nn.ReLU()

In [14]:
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)
torch.nn.init.xavier_normal_(layer3.weight)
torch.nn.init.xavier_normal_(layer4.weight)
torch.nn.init.xavier_normal_(layer5.weight)

Parameter containing:
tensor([[-0.0038, -0.0342, -0.0981,  0.1425,  0.0082,  0.1110, -0.0675,  0.1191,
          0.1155,  0.0255, -0.0960, -0.0383, -0.0750,  0.0915,  0.0563, -0.1904,
          0.1326, -0.1942,  0.0743,  0.0410, -0.1687,  0.0675,  0.0055, -0.0704,
         -0.0233, -0.0545,  0.0461, -0.0881,  0.0453, -0.1822, -0.0745, -0.0085,
          0.1890,  0.0769, -0.2320,  0.0369,  0.0607, -0.0749, -0.0681, -0.0037,
          0.0896, -0.1216,  0.1988,  0.0275,  0.1064,  0.0052, -0.0240,  0.0368,
         -0.0244, -0.0093,  0.0193, -0.0164, -0.0504,  0.1657,  0.1024, -0.0050,
          0.1750,  0.0291,  0.0836, -0.1180, -0.0313,  0.0018,  0.0817,  0.0122,
         -0.0409, -0.0476, -0.0164,  0.0011, -0.0099,  0.1416, -0.0318, -0.0021,
          0.0646,  0.1394, -0.0224, -0.0131,  0.0458, -0.0824, -0.0275, -0.0920,
          0.0573, -0.0216,  0.2241, -0.1000,  0.1695, -0.1352, -0.0401, -0.0714,
          0.0664, -0.0232,  0.0525,  0.0978, -0.0796,  0.0041, -0.0255, -0.0377,
      

In [15]:
model = torch.nn.Sequential(layer1, relu,
                            layer2, relu,
                            layer3, relu,
                            layer4, relu,
                            layer5).cuda()

In [16]:
learning_rate = 0.001
loss = torch.nn.L1Loss().cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [17]:
training_epochs = 30
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X,Y in data_loader:
        X = X.cuda()
        Y = Y.cuda()
        
        optimizer.zero_grad()
        hypothesis = model(x_train)
        cost = loss(hypothesis,y_train)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 149.518234253
Epoch: 0002 cost = 61.917804718
Epoch: 0003 cost = 51.816825867
Epoch: 0004 cost = 46.658767700
Epoch: 0005 cost = 43.588291168
Epoch: 0006 cost = 41.638435364
Epoch: 0007 cost = 39.599609375
Epoch: 0008 cost = 38.747509003
Epoch: 0009 cost = 37.694461823
Epoch: 0010 cost = 36.926967621
Epoch: 0011 cost = 35.463272095
Epoch: 0012 cost = 35.594978333
Epoch: 0013 cost = 33.713718414
Epoch: 0014 cost = 32.901962280
Epoch: 0015 cost = 32.679630280
Epoch: 0016 cost = 31.837171555
Epoch: 0017 cost = 31.588470459
Epoch: 0018 cost = 30.805040359
Epoch: 0019 cost = 30.498851776
Epoch: 0020 cost = 29.634656906
Epoch: 0021 cost = 29.023502350
Epoch: 0022 cost = 28.713876724
Epoch: 0023 cost = 27.965167999
Epoch: 0024 cost = 27.744682312
Epoch: 0025 cost = 27.127540588
Epoch: 0026 cost = 26.836566925
Epoch: 0027 cost = 26.254148483
Epoch: 0028 cost = 25.967641830
Epoch: 0029 cost = 25.996425629
Epoch: 0030 cost = 25.052404404


In [18]:
predict=model(x_test).cpu().detach().numpy()
submit['predict']=predict
submission=submit.astype(np.int32)
submission.to_csv("submission.csv",mode='w',header=True,index=False)